<a href="https://colab.research.google.com/github/Sergey-Kit/itmo_dl_nlp_course/blob/hw_7/itmo_dl_nlp_course_dz_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question Answering на русском языке

##### 1. Установка зависимостей

In [ ]:
!pip install datasets=="2.14.6" transformers=="4.35.0" accelerate=="0.24.1"

In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer, pipeline

In [4]:
RANDOM_STATE = 42

##### 2. Загрузка датасета

In [5]:
dataset = load_dataset("sberquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

##### 3. Делим выборку

In [60]:
train_data = dataset['train'].select(range(4000))
test_data = dataset['test'].select(range(500))

##### 4. Токенизация


In [109]:
model_checkpoint = "DeepPavlov/rubert-base-cased"

In [110]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [111]:
max_length = 512
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [112]:
train_dataset = train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data.column_names,
)
len(train_data), len(train_dataset)

(4000, 4000)

In [113]:
test_dataset = test_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=test_data.column_names,
)
len(test_data), len(test_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

(500, 500)

##### 5. Параметры обучения

In [114]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [115]:
args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
    fp16=True,
    seed=RANDOM_STATE,
)

In [116]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [117]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.274000,3.285635
2,1.635700,3.689395


TrainOutput(global_step=1000, training_loss=2.22153857421875, metrics={'train_runtime': 426.993, 'train_samples_per_second': 18.736, 'train_steps_per_second': 2.342, 'total_flos': 2090374053888000.0, 'train_loss': 2.22153857421875, 'epoch': 2.0})

##### 5.Сравнение

In [118]:
metric = load_metric("squad")

def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    start_preds, end_preds = predictions
    start_labels, end_labels = labels

    start_preds = np.argmax(start_preds, axis=1)
    end_preds = np.argmax(end_preds, axis=1)

    formatted_predictions = []
    for idx, (start, end) in enumerate(zip(start_preds, end_preds)):
        pred_text = tokenizer.decode(test_dataset[idx]['input_ids'][start:end+1],
                                     skip_special_tokens=True)
        formatted_predictions.append({"id": idx, "prediction_text": pred_text})

    references = [{"id": idx, "answers": dataset["validation"][idx]['answers']} for idx in range(len(test_dataset))]

    return metric.compute(predictions=formatted_predictions, references=references)

trainer_base = Trainer(
    model=AutoModelForQuestionAnswering.from_pretrained(model_checkpoint),
    compute_metrics=compute_metrics,
    eval_dataset=test_dataset,
)

base_metrics = trainer_base.evaluate()

trainer.compute_metrics = compute_metrics

fine_tuned_metrics = trainer.evaluate()

results_df = pd.DataFrame({
    "Metric": ["Loss", "Exact Match", "F1 Score"],
    "Base Model": [base_metrics['eval_loss'], base_metrics['eval_exact_match'], base_metrics['eval_f1']],
    "Fine-tuned Model": [fine_tuned_metrics['eval_loss'], fine_tuned_metrics['eval_exact_match'], fine_tuned_metrics['eval_f1']],
})


display(results_df)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Metric,Base Model,Fine-tuned Model
0,Loss,6.334570,3.285635
1,Exact Match,0.000000,0.000000
2,F1 Score,0.482841,1.237469


##### 6.Валидация

In [119]:
trainer.model.to('cpu')

question_answering_pipeline = pipeline("question-answering", model=trainer.model, tokenizer=tokenizer)

context = "Около Порт-Артура ночью на 27 января 1904 года начались первые боевые столкновения Русско-японской войны, когда японские корабли выпустили торпеды по русским военным кораблям, стоявшим на внешнем рейде Порт-Артура. При этом были серьёзно повреждены броненосцы Ретвизан и Цесаревич , а также крейсер Паллада . Оставшиеся корабли предприняли две попытки вырваться из порта, но обе оказались неудачными. Нападение японцев было совершено без объявления войны и было осуждено большинством стран мирового сообщества. Только Великобритания, являвшаяся в то время союзником Японии, праздновала нападение как великое деяние ."
questions = [
    "Кто праздновал нападение на Российский флот в 1904 году?",
    "Где были первые столкновения Русско-японской войны 1904 года?",
    "Какая дата начала Русско-японской войны?",
    "Где началась Русско-японской война?"
]

for question in questions:
    answer = question_answering_pipeline(question=question, context=context)
    print("\nВопрос:", question)
    print("Ответ:", answer['answer'])


Вопрос: Кто праздновал нападение на Российский флот в 1904 году?
Ответ: Великобритания

Вопрос: Где были первые столкновения Русско-японской войны 1904 года?
Ответ: Около Порт-Артура

Вопрос: Какая дата начала Русско-японской войны?
Ответ: 27 января 1904 года

Вопрос: Где началась Русско-японской война?
Ответ: Около Порт-Артура ночью на 27 января 1904 года
